# Objetivo

**Aluna:** Tathiana de Souza Martins
**Email:** tsm2@cesar.school

Este modelo foi desenvolvido para atestar se um determinado produto está superfaturado ou não. Pensado como uma ferramenta para auxiliar auditores ou analistas do TCE e/ou TCU a identificar produtos superfaturados em alguma obra pública. O projeto segue as diretrizes do projeto final do módulo Assuntos Contemporâneos da Pós-graduação de Engenharia e Análises de Dados do CESAR School, projeto este proposto pelo professor Silvan Ferreira.

O aplicativo usa um sistema RAG utilizando a biblioteca LangChain e SerpiAPI para extrair os preços de um determinado produto, o Langchain Tools para criar funções personalizadas a serem usadas pelo modelo da OpenAI disponível abertamente, "gpt-4o-mini".

Para utilizar é necessário apenas escrever o produto que o usuário deseja pesquisar o valor e o valor de corte, ou a pergunta se o valor está superfaturado ou não. O resultado é uma lista do produto pesquisado com os respectivos preços e link da loja e uma resposta estilo ChatGPT se o produto é superfaturado ou não.

## Baixando as Bibliotecas

In [ ]:
!pip install --upgrade --quiet  langchain-google-community
!pip install --quiet langchain
!pip install --quiet google-search-results
!pip install --quiet google-search-results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install langchain_openai langchain_core langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
## Importando as Bibliotecas

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.utilities import SerpAPIWrapper
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
os.environ['SERPAPI_API_KEY'] =  getpass.getpass() # Use a sua chave SerpAPI
os.environ['GOOGLE_API_KEY'] = getpass.getpass() # Use a sua chave Google API


## Baixando o modelo da OpenAI GPT 4o mini

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Funções e Tools

In [ ]:
@tool
def fetch_google_shopping(query: str, max_results: int = 10):
    """Busca no Google Shopping para comparar preços de produtos.

    Args:
        query: A pesquisa a ser realizada no Google Shopping.
        max_results: Número máximo de resultados desejados.
    """
    search = SerpAPIWrapper(
        params={
            "gl": "br",           # Geolocalização: Brasil
            "hl": "pt-BR",        # Idioma: Português do Brasil
            "tbm": "shop",        # Tipo de busca: Google Shopping
            "num": max_results,   # Número de resultados desejados
        }
    )
    result = search.run(query)
    return result

@tool
def calcula_valor_min(query_result):
    """Calcula o valor mínimo entre os resultados da pesquisa.

    Args:
        query_result: Resultados da pesquisa.

    Returns:
        O menor valor encontrado nos resultados.
    """
    prices = [item.get('extracted_price') for item in query_result if item.get('extracted_price') is not None]
    if not prices:
        return "Nenhum preço disponível"
    return min(prices)

@tool
def superfaturado(price: float, min_price: float) -> str:
    """Calcula se o valor de um item é superfaturado.

    Args:
        price: Preço do item.
        min_price: Preço mínimo encontrado.

    Returns:
        'Superfaturado' ou 'Não superfaturado'.
    """
    if price >= min_price * 2:
        return 'Superfaturado'
    else:
        return 'Não superfaturado'


# Lista de ferramentas
tools = [fetch_google_shopping, calcula_valor_min, superfaturado]


## Fazendo a query e a resposta

In [ ]:
agent_executor = create_react_agent(llm, tools)

response = agent_executor.invoke({
    "messages": [
        HumanMessage(content="Pesquise preço para Desktop i3 8gb hd 500gb, e verifique se há um preço menor do que R$2645.52")
    ]
})


[HumanMessage(content='Pesquise preço para Desktop i3 8gb hd 500gb, e verifique se há um preço menor do que R$2645.52', additional_kwargs={}, response_metadata={}, id='5cc76b3a-579e-4d53-a338-d42a46badf04'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W82GxWAoW05urzptNL7ysA19', 'function': {'arguments': '{"query":"Desktop i3 8gb hd 500gb"}', 'name': 'fetch_google_shopping'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 227, 'total_tokens': 253, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64b5a464-3039-4bee-8f12-53fcd0ab7501-0', tool_calls=[{'name': 'fetch_google_shopping', 'args': {'query': 'Deskt

In [ ]:
print(response["messages"][-1].content)

Os preços encontrados para desktops com as especificações solicitadas são:

1. **Computador Completo Intel Core I3 8GB HD 500GB Monitor Wifi**
   - Preço: R$ 881,84
   - [Link para compra](https://www.casasbahia.com.br/computador-completo-intel-core-i3-8gb-hd-500gb-monitor-wifi-1536708788/p/1536708788?utm_medium=Cpc&utm_source=google_freelisting&IdSku=1536708788&idLojista=145309&tipoLojista=3P&srsltid=AfmBOoo6Ufj0l2MDylYladsCpskzRiWsWF0PtxRX8T5k6l5YRgN1VMGi-Rk)

2. **Computador Desktop Intel Core I3, 8GB RAM, SSD 240GB, HDMI, Windows 10 Pro 64 Bits**
   - Preço: R$ 490,99
   - [Link para compra](https://www.kabum.com.br/produto/569256/computador-desktop-intel-core-i3-8gb-ram-ssd-240gb-hdmi-windows-10-pro-64-bits?srsltid=AfmBOorQ7Of8mpjKUGCYOrtJCYeiPfdsfWX8NbNZEIjIbdj7aiEyL1kESOE)

3. **Computador Desktop, Intel Core I3 4ª Geração, 8GB RAM, HD SSD 480GB**
   - Preço: R$ 862,53
   - [Link para compra](https://shopee.com.br/product/275151625/22697319464?gads_t_sig=VTJGc2RHVmtYMTlxTFVSVVRr

## Conclusão

Geralmente os valores de produtos superfaturados em obras públicas possui o valor triplicado de seu valor médio ou mínimo de mercado. Por isso, a comparação de preços foi feita com o valor R$2645.52

A resposta gerada pelo modelo foi satisfatória e dentro do esperado, o modelo conseguiu determinar corretamente que o produto estava superfaturado.

**Fontes:**


* https://python.langchain.com/docs/integrations/tools/google_serper/#as-part-of-a-self-ask-with-search-chain

* https://python.langchain.com/docs/integrations/providers/serpapi/


* https://python.langchain.com/docs/integrations/providers/google/#3rd-party-integrations

* https://python.langchain.com/docs/integrations/tools/google_search/

* https://python.langchain.com/docs/how_to/tools_builtin/#customizing-default-tools

* https://serpapi.com/

* https://python.langchain.com/docs/integrations/providers/google/#google-search

* https://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search
